    0 = neutral,                0 - нейтральність
    1 = anger,                  1 - злість
    2 = contempt,               2 - зневага
    3 = disgust,                3 - огида 
    4 = fear,                   4 - страх
    5 = happy,                  5 - радість 
    6 = sadness,                6 - сум 
    7 = surprise                7 - здивування 

In [1]:
import os
from shutil import copy, move

import numpy as np
import pandas as pd
import dlib
import cv2

In [2]:
emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]

if not os.listdir('sorted_data'):
    print('No emotions within sorted_data detected. Creating new')
    for em in emotions:
        os.makedirs('sorted_data/'+em)

In [3]:
encoded_emotions = {t: em for t, em in enumerate(emotions)}

In [4]:
encoded_emotions

{0: 'neutral',
 1: 'anger',
 2: 'contempt',
 3: 'disgust',
 4: 'fear',
 5: 'happy',
 6: 'sadness',
 7: 'surprise'}

Get all participants of the dataset

In [5]:
img_path = 'src_images'
annotations_path = 'src_labels'
destination = 'sorted_data'

raw_collector = []

participants = os.listdir(img_path)
labels = os.listdir(annotations_path)
# Map emotions with folder names
emo_map = dict(zip(emotions, [f'{k:03}' for k in range(7)]))

for person in sorted(labels):
    for emotion in os.listdir(os.path.join(annotations_path, person)):
        emotion_path = os.path.join(annotations_path, person, emotion)
        if os.listdir(emotion_path):
            with open(os.path.join(emotion_path, os.listdir(emotion_path)[0]), 'r') as f:
                sample = [os.listdir(emotion_path)[0], f.read()]
                raw_collector.append(sample)


In [6]:
collector = []

for filename, emotion in raw_collector:
    file_metadata = filename.split('_')
    emotion = float(emotion.replace(' ', '').replace('e+00\n', ''))
    collector.append([file_metadata, emotion])

##### Create a backup

In [7]:
with open('backup_0.csv', 'w') as f:
    f.write(str(collector).replace('.0],', '\n').replace('[', '').replace(']', ''))

#### Move images to appropriate folders

In [8]:
for [person, collection, img, ll], label in collector:
    # Such a shit :)
    assert label > 0, f'{person, collection} has neutral ({label}) label'
    
    path_to_img_folder = os.path.join(img_path, person, collection)
    
    images = sorted(os.listdir(path_to_img_folder))
    # First 20% of images are neutral. The last (suppose 50%) are emotional.
    n = len(images)
    for i, image in enumerate(images, 1):
        img_with_path = os.path.join(path_to_img_folder, image)
        # Mark first 20% of images as neutral
        if i / n <= 0.2:
            copy(img_with_path, os.path.join(destination, encoded_emotions[0], image))
        elif i / n >= 0.4:
            copy(img_with_path, os.path.join(destination, encoded_emotions[int(label)], image))
        # Mark last 50% of images as not neutral
        #print(img_with_path, label, encoded_emotions[int(label)], round(100 * i / n, 2))
    

Now go to folders & throw out some images manually (especially from the neutral folder)



After that pass all the images through the dlib keypoint detector to get all the facial points

In [9]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../face_landmarks.dat')


def shape_to_np(shape, dtype="int", landmarks_count=68):
    point = np.zeros((landmarks_count, 2), dtype=dtype)

    for i in range(landmarks_count):
        point[i] = (shape.part(i).x, shape.part(i).y)
    return point


def get_normalized_coords(shape, w, h):
    face_landmarks = np.zeros(shape.shape)
    
    x_max = shape[:, 0].max()
    y_max = shape[:, 1].max()
    x_min = shape[:, 0].min()
    y_min = shape[:, 1].min()
    
    face_landmarks[:, 0] = (x_max - shape[:, 0]) / (x_max - x_min)
    face_landmarks[:, 1] = (y_max - shape[:, 1] / (y_max - y_min))
    
    return face_landmarks


def get_radius_vector(norm_shape, x_c, y_c):
    x, y = np.split(np.array(norm_shape), 2, axis=1)
    return np.sqrt(np.square(x - x_c) + np.square(y - y_c))


def get_angle(norm_shape):
    x, y = np.split(np.array(norm_shape), 2, axis=1)
    return np.arctan([x / y])


def get_face_landmarks(img_name, img_array, predictor):
    gray_image = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    h, w = img_array.shape[:2]
    rects = detector(gray_image, 1)
#     rad_vectors = []
#     face_landmarks = []
    for i, rect in enumerate(rects):
        # remove this "if" in case, this is not training exxample
        if i > 1:
            print(f'{img_name} seems to fail')
            break
        face_detector = predictor(gray_image, rect)
        shape = shape_to_np(face_detector)
        shape = get_normalized_coords(shape, w, h)
        radius_vector = get_radius_vector(shape, w, h)
        angle = get_angle(shape)

        return shape, radius_vector, angle




In [10]:
#!pip install atpbar
from atpbar import atpbar

In [11]:
jsonlike = []
for emotion in atpbar(emotions, name='images'):
    for image in atpbar(os.listdir(os.path.join(destination, emotion)), name=emotion):
        imgpath = os.path.join(destination, emotion, image)
        try:
            img_array = cv2.imread(imgpath)
            face_landmarks, rad_vectors, angles = get_face_landmarks(image, img_array, predictor)
        except:
            print(f'failed on {imgpath}')
            continue
        
        current_image_collector = {'image': image, 'emotion': emotion}

        
        #print(face_landmarks[0])
        
        for i, [x, y] in enumerate(face_landmarks):
            current_image_collector.update({f'x_{i}':x})
            current_image_collector.update({f'y_{i}':y})
            current_image_collector.update({f'r_{i}':rad_vectors[i]})
            current_image_collector.update({f'a_{i}':y})

        jsonlike.append(current_image_collector)

        
        


VBox()

failed on sorted_data/neutral/.DS_Store


In [24]:
df2 = pd.DataFrame(jsonlike)

In [25]:
df2.head()

,a_0,a_1,a_10,a_11,a_12,a_13,a_14,a_15,a_16,a_17,...,y_61,y_62,y_63,y_64,y_65,y_66,y_67,y_7,y_8,y_9
0,382.156250,382.017857,381.397321,381.504464,381.625000,381.754464,381.888393,382.022321,382.151786,382.223214,...,381.642857,381.633929,381.642857,381.633929,381.642857,381.633929,381.642857,381.312500,381.290179,381.308036
1,381.653846,381.527473,380.972527,381.054945,381.164835,381.291209,381.423077,381.554945,381.686813,381.763736,...,381.241758,381.236264,381.241758,381.225275,381.236264,381.230769,381.236264,380.906593,380.895604,380.912088
2,345.092683,344.960976,344.380488,344.458537,344.560976,344.682927,344.804878,344.926829,345.058537,345.234146,...,344.643902,344.639024,344.643902,344.619512,344.643902,344.639024,344.643902,344.331707,344.312195,344.326829
3,405.034483,404.896552,404.306034,404.383621,404.491379,404.612069,404.750000,404.887931,405.030172,405.163793,...,404.612069,404.603448,404.603448,404.581897,404.603448,404.599138,404.607759,404.271552,404.250000,404.258621
4,419.119522,418.996016,418.426295,418.521912,418.633466,418.745020,418.864542,418.980080,419.099602,419.219124,...,418.693227,418.681275,418.685259,418.665339,418.665339,418.657371,418.669323,418.350598,418.326693,418.350598


In [26]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4766 entries, 0 to 4765
Columns: 274 entries, a_0 to y_9
dtypes: float64(204), object(70)
memory usage: 10.0+ MB


In [15]:
df2.describe()

,a_0,a_1,a_10,a_11,a_12,a_13,a_14,a_15,a_16,a_17,...,y_61,y_62,y_63,y_64,y_65,y_66,y_67,y_7,y_8,y_9
count,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,...,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000,4766.000000
mean,392.546838,392.414520,391.838637,391.928395,392.039999,392.166031,392.297604,392.429416,392.563052,392.644401,...,392.112430,392.107203,392.113405,392.094395,392.076526,392.068707,392.075532,391.767529,391.748983,391.771020
std,29.936782,29.938205,29.939801,29.941281,29.941792,29.941290,29.940685,29.940046,29.939464,29.937673,...,29.938708,29.939093,29.938873,29.934585,29.930092,29.929983,29.929916,29.939995,29.939983,29.940006
min,310.226131,310.085427,309.487437,309.547739,309.638191,309.758794,309.884422,310.015075,310.160804,310.331658,...,309.804020,309.798995,309.798995,309.778894,309.793970,309.793970,309.804020,309.462312,309.437186,309.447236
25%,369.990577,369.859379,369.304087,369.385714,369.485714,369.605952,369.738245,369.873459,370.010714,370.087805,...,369.565477,369.564258,369.569499,369.562054,369.562806,369.553688,369.560976,369.231971,369.216346,369.236779
50%,391.222456,391.092632,390.518098,390.613328,390.728173,390.847228,390.971460,391.098393,391.227814,391.299393,...,390.769751,390.763449,390.767651,390.756952,390.739387,390.729692,390.737521,390.435016,390.412721,390.439066
75%,416.247913,416.120253,415.576030,415.664474,415.775954,415.898507,416.021431,416.144229,416.268781,416.371856,...,415.870200,415.870200,415.876492,415.812959,415.749257,415.742980,415.748318,415.505395,415.489070,415.510810
max,483.215686,483.084967,482.503268,482.598039,482.712418,482.839869,482.970588,483.098039,483.222222,483.320261,...,482.807190,482.807190,482.807190,482.758170,482.787582,482.787582,482.787582,482.437908,482.418301,482.437908


In [27]:
df2 = df2.sample(frac=1)

In [28]:
df2.head()

,a_0,a_1,a_10,a_11,a_12,a_13,a_14,a_15,a_16,a_17,...,y_61,y_62,y_63,y_64,y_65,y_66,y_67,y_7,y_8,y_9
1496,416.955752,416.823009,416.238938,416.331858,416.451327,416.579646,416.712389,416.845133,416.982301,417.039823,...,416.486726,416.482301,416.486726,416.482301,416.469027,416.460177,416.469027,416.172566,416.150442,416.172566
1314,445.074219,444.925781,444.324219,444.402344,444.507812,444.640625,444.781250,444.925781,445.070312,445.140625,...,444.636719,444.632812,444.640625,444.621094,444.636719,444.632812,444.632812,444.269531,444.257812,444.273438
834,365.200873,365.069869,364.489083,364.576419,364.694323,364.820961,364.951965,365.091703,365.222707,365.331878,...,364.790393,364.786026,364.790393,364.772926,364.786026,364.786026,364.786026,364.423581,364.401747,364.423581
3839,432.040323,431.899194,431.346774,431.431452,431.536290,431.653226,431.782258,431.915323,432.060484,432.125000,...,431.584677,431.580645,431.584677,431.556452,431.528226,431.516129,431.520161,431.266129,431.254032,431.286290
796,368.728814,368.598870,367.988701,368.073446,368.186441,368.310734,368.451977,368.587571,368.717514,368.819209,...,368.299435,368.288136,368.293785,368.276836,368.299435,368.288136,368.299435,367.932203,367.909605,367.926554


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle

from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [30]:
label = df2['emotion']

In [31]:
le = LabelEncoder()
y = le.fit_transform(label)

In [32]:
df2 = df2.drop(['emotion', 'image'], axis=1)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(df2, y, test_size=0.2)

In [34]:
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()

/home/yurii/Work/rd/emo/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/yurii/Work/rd/emo/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [35]:
X_train.shape, X_test.shape

((3812, 272), (954, 272))

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
print(model)

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
pickle.dump(model, open('xgb_model.pickle', 'wb'))

In [ ]:
restored = pickle.load(open("xgb_model.pickle", "rb"))

In [ ]:
#restored2 = pickle.load(open("xgb_with_rv.pickle", "rb"))

In [ ]:
X_sample = X_test[-1]

### Restoring the original point coordinates

In [ ]:
x_c = 100
y_c = 100
w = 200
h = 200

In [ ]:
%time

shape = []
for k in range(0, len(X_sample), 3):
    x = x_c - w * X_sample[k+1] 
    y = y_c - h * X_sample[k+2]
    shape.append([x, y])
x_s = [s[0] for s in shape]
y_s = [s[1] for s in shape]

plt.scatter(x_s, y_s)

In [ ]:
%time

rand_ar = np.array(X_sample).reshape(68,3)
rand_ar = np.delete(rand_ar, obj=0, axis=1)

rand_ar[:,0] = - x_c + w * rand_ar[:,0]
rand_ar[:,1] = - y_c + h * rand_ar[:,1]

plt.scatter(rand_ar[:,0], rand_ar[:,1])

In [ ]:
def restore_coordinates(norm_shape, x_c, y_c, w, h):
    shape = np.array(norm_shape).reshape(68,3)
    shape = np.delete(shape, obj=0, axis=1)

    shape[:,0] = - x_c + w * rand_ar[:,0]
    shape[:,1] = - y_c + h * rand_ar[:,1]
    return shape

In [ ]:
pred = model.predict(X_sample.reshape([1, 204]))

In [ ]:
encoded_emotions[int(pred)]